 <!-- ---
title: "Client Report - [Project 2: Late flights and missing data]"
subtitle: "Course DS 250"
author: "[Joshua Ludwig]"
format:
  html:
    self-contained: true
    page-layout: full
    title-block-banner: true
    toc: true
    toc-depth: 3
    toc-location: body
    number-sections: false
    html-math-method: katex
    code-fold: true
    code-summary: "Show the code"
    code-overflow: wrap
    code-copy: hover
    code-tools:
        source: false
        toggle: true
        caption: See code
    
--- -->




## Elevator pitch

_In this project I learned how to create new columns with new infomration and how to organize it to be more readable an dunderstandable to someone unfamiliar with altair. In this project I calcualted which airport has the wrost delays, what month is the best to travel with the least delays, and more. _


In [ ]:
#| label: libraries
#| include: false
import pandas as pd
import altair as alt
import numpy as np

from IPython.display import Markdown
from IPython.display import display
from tabulate import tabulate

In [ ]:
missing_flights = pd.read_json("flights_missing.json")

__Highlight the grand questions__

## GRAND QUESTION 1

__Which airport has the worst delays?__

_The airport with the worst delay is SFO with a proportional delay of 34. This means that on average every flight has a delay of 34 hours._


In [ ]:
#| label: GQ1
missing_flights['hours_delayed_total'] = missing_flights['minutes_delayed_total'] / 60

missing_flights['proportion_delayed'] = missing_flights['num_of_delays_total'] / missing_flights['num_of_flights_total']

# Find the airport with the highest total hours of delay
worst_airport = missing_flights.loc[missing_flights['hours_delayed_total'].idxmax()]

chart1 = missing_flights.groupby('airport_code').sum().filter(items=['num_of_flights_total', 'num_of_delays_total', 'proportion_delayed', 'hours_delayed_total'])

chart1

## GRAND QUESTION 2

__What is the best month to fly if you want to avoid delays of any length?__

_I found that the best month to fly without any delays is September. During September there are nearly 12 delays per flight._


In [ ]:
#| label: GQ2
remove_month = missing_flights[missing_flights["month"] != "n/a"]
# This removes any n/a in the "month" column

remove_month['month'] = remove_month['month'].replace('Febuary', 'February')

month_order = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]

remove_month['month'] = pd.Categorical(remove_month['month'], categories=month_order, ordered=True)

chart2 = remove_month.groupby(by='month').sum().filter(items=['num_of_flights_total', 'num_of_delays_total', 'proportion_delayed'])
#%%
chart2

## GRAND QUESTION 3

__Create a new column that calculates the total number of flights delayed by weather (both severe and mild).__

_The table created shows the number of delays calcualted in total at each airport._


In [ ]:
#| label: GQ3
# Step 1: Replace missing values in the "num_of_delays_late_aircraft" column with the mean
mean_late_aircraft = missing_flights['num_of_delays_late_aircraft'].mean()
missing_flights['num_of_delays_late_aircraft'].fillna(mean_late_aircraft, inplace=True)

# Step 2: Create a new column "num_of_delays_weather_total" initialized with zeros
missing_flights['num_of_delays_weather_total'] = 0

# Step 3: Apply the rules to calculate the values for "num_of_delays_weather_total" column
# Rule 1: 100% of delayed flights in the Weather category are due to weather
missing_flights.loc[missing_flights['num_of_delays_weather'].notnull(), 'num_of_delays_weather_total'] = missing_flights['num_of_delays_weather']

# Rule 2: 30% of all delayed flights in the Late-Arriving category are due to weather
missing_flights['num_of_delays_weather_total'] += 0.3 * missing_flights['num_of_delays_late_aircraft']

chart3 = missing_flights.num_of_delays_weather_total.head()
chart3 = missing_flights.groupby('airport_code').sum().filter(items=['num_of_flights_total', 'num_of_delays_total', 'num_of_delays_weather_total', 'hours_delayed_total'])
# %%
chart3

## GRAND QUESTION 4

__Create a barplot showing the proportion of all flights that are delayed by weather at each airport.__

_The barplot shows the proportion of delays caused by the weather at each airport. It appears Salt Lake City airport has the highest proportion of delays caused by weather compared to other airports._


In [ ]:
#| label: GQ4
weather_df = missing_flights[['airport_code', 'num_of_delays_weather_total', 'num_of_flights_total']]

# Calculate the proportion of flights delayed by weather
weather_df['proportion_delayed_by_weather'] = weather_df['num_of_delays_weather_total'] / weather_df['num_of_flights_total']

# Create a barplot of the proportion of flights delayed by weather
barplot = alt.Chart(weather_df).mark_bar().encode(
    x='airport_code',
    y='proportion_delayed_by_weather',
    tooltip=['airport_code', 'proportion_delayed_by_weather']
).properties(
    title='Proportion of Flights Delayed by Weather at Each Airport',
    width=600,
    height=400
)

# Show the barplot
barplot.show()


## GRAND QUESTION 5

__Fix all of the varied missing data types in the data to be consistent (all missing values should be displayed as “NaN”)__

_At index 921, the data in month column is replaced with "NaN"_


In [ ]:
#| label: GQ5
missing_flights.replace('n/a', np.nan, inplace=True)
missing_flights

## APPENDIX A (Additional Python Code)


In [ ]:
#%%
import pandas as pd
import altair as alt
import numpy as np
from scipy import stats


#%%
missing_flights = pd.read_json("flights_missing.json")

missing_flights['hours_delayed_total'] = missing_flights['minutes_delayed_total'] / 60

missing_flights['proportion_delayed'] = missing_flights['num_of_delays_total'] / missing_flights['num_of_flights_total']
# Find the airport with the highest total hours of delay
worst_airport = missing_flights.loc[missing_flights['hours_delayed_total'].idxmax()]

chart1 = missing_flights.groupby('airport_code').sum().filter(items=['num_of_flights_total', 'number_of_delays_total', 'proportion_delayed', 'hours_delayed_total'])
#%%
chart1




#%%
remove_month = missing_flights[missing_flights["month"] != "n/a"]
# This removes any n/a in the "month" column

remove_month['month'] = remove_month['month'].replace('Febuary', 'February')

month_order = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]

remove_month['month'] = pd.Categorical(remove_month['month'], categories=month_order, ordered=True)

chart2 = remove_month.groupby(by='month').sum().filter(items=['num_of_flights_total', 'num_of_delays_total', 'proportion_delayed'])
#%%
chart2





# %%

# Step 1: Replace missing values in the "num_of_delays_late_aircraft" column with the mean
mean_late_aircraft = missing_flights['num_of_delays_late_aircraft'].mean()
missing_flights['num_of_delays_late_aircraft'].fillna(mean_late_aircraft, inplace=True)

# Step 2: Create a new column "num_of_delays_weather_total" initialized with zeros
missing_flights['num_of_delays_weather_total'] = 0

# Step 3: Apply the rules to calculate the values for "num_of_delays_weather_total" column
# Rule 1: 100% of delayed flights in the Weather category are due to weather
missing_flights.loc[missing_flights['num_of_delays_weather'].notnull(), 'num_of_delays_weather_total'] = missing_flights['num_of_delays_weather']

# Rule 2: 30% of all delayed flights in the Late-Arriving category are due to weather
missing_flights['num_of_delays_weather_total'] += 0.3 * missing_flights['num_of_delays_late_aircraft']

chart3 = missing_flights.num_of_delays_weather_total.head()
# %%
chart3




# %%
# Create a new DataFrame with the weather variable
weather_df = missing_flights[['airport_code', 'num_of_delays_weather_total', 'num_of_flights_total']]

# Calculate the proportion of flights delayed by weather
weather_df['proportion_delayed_by_weather'] = weather_df['num_of_delays_weather_total'] / weather_df['num_of_flights_total']

# Create a barplot of the proportion of flights delayed by weather
barplot = alt.Chart(weather_df).mark_bar().encode(
    x='airport_code',
    y='proportion_delayed_by_weather',
    tooltip=['airport_code', 'proportion_delayed_by_weather']
).properties(
    title='Proportion of Flights Delayed by Weather at Each Airport',
    width=600,
    height=400
)

# Show the barplot
barplot.show()




# %%

missing_flights.replace('n/a', np.nan, inplace=True)
missing_flights